# Tester des Modèles d'Embedding
---

Ce notebook accompagne la vidéo tutoriel sur le test et l'utilisation de modèles d'embedding avec Hugging Face et Mistral AI dans le cadre du projet RAG (Retrieval-Augmented Generation) pour la mairie de Trifouillis-sur-Loire.

### Objectif : Explorer comment générer des représentations vectorielles (embeddings) de textes pour améliorer la recherche d'informations pertinentes.

#### Outils utilisés :

*   Hugging Face: Pour accéder à des modèles d'embedding open source via la bibliothèque sentence-transformers.
*   Mistral AI: Pour utiliser des modèles d'embedding performants via leur API serverless.

#### Objectifs :
En comparant la similarité entre vecteur_requête et les vecteur_document disponibles, le système RAG peut identifier le document le plus pertinent pour répondre à la question, même si les mots exacts ne sont pas identiques.



In [ ]:
# @title Installation des bibliothèques
!pip install sentence-transformers numpy -q
print("Bibliothèques installées !")

In [ ]:
# @title Chargement du modèle et vectorisation de textes
from sentence_transformers import SentenceTransformer
import numpy as np

# Charger le modèle depuis Hugging Face Hub
print("Chargement du modèle Sentence Transformer...")
model_hf = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Modèle chargé !")

# Textes d'exemple (simulant une question et un extrait de document)
textes_hf = [
    "Quelles sont les heures d'ouverture de la mairie ?", # Simulation question citoyen
    "La mairie de Trifouillis-sur-Loire est ouverte du lundi au vendredi de 8h30 à 17h00.", # Simulation info document
    "Le marché hebdomadaire a lieu tous les samedis matin sur la place centrale." # Info non pertinente
]

# Générer les embeddings
print("\nGénération des embeddings...")
embeddings_hf = model_hf.encode(textes_hf)

print("Embeddings générés !")
print("Shape du premier embedding :", embeddings_hf[0].shape) # Montre la dimension du vecteur
# Optionnel : Afficher les embeddings (peut être très long)
# print("\nEmbeddings :")
# print(embeddings_hf)

In [ ]:
# @title Calcul de la similarité cosinus

def cosine_similarity(vec1, vec2):
  """Calcule la similarité cosinus entre deux vecteurs."""
  dot_product = np.dot(vec1, vec2)
  norm_vec1 = np.linalg.norm(vec1)
  norm_vec2 = np.linalg.norm(vec2)
  if norm_vec1 == 0 or norm_vec2 == 0:
    return 0 # Éviter la division par zéro
  return dot_product / (norm_vec1 * norm_vec2)

# Embedding de la question
embedding_question = embeddings_hf[0]

# Embeddings des documents potentiels
embedding_doc1 = embeddings_hf[1]
embedding_doc2 = embeddings_hf[2]

# Calcul des similarités
similarity_q_doc1 = cosine_similarity(embedding_question, embedding_doc1)
similarity_q_doc2 = cosine_similarity(embedding_question, embedding_doc2)

print(f"Question : '{textes_hf[0]}'")
print("-" * 30)
print(f"Document 1 : '{textes_hf[1]}'")
print(f"Similarité avec la question : {similarity_q_doc1:.4f}")
print("-" * 30)
print(f"Document 2 : '{textes_hf[2]}'")
print(f"Similarité avec la question : {similarity_q_doc2:.4f}")
print("-" * 30)

if similarity_q_doc1 > similarity_q_doc2:
    print("\nConclusion : Le document 1 est sémantiquement plus proche de la question.")
else:
    print("\nConclusion : Le document 2 est sémantiquement plus proche de la question.")

In [ ]:
# @title Installation de la bibliothèque Mistral AI
!pip install mistralai
print("Bibliothèque Mistral AI installée !")

In [ ]:
import os
from mistralai import Mistral

# Remplacez par votre vraie clé API
api_key = "StthE16bt5IReMHPxgjbwtQLGggjOGtq"

# Initialisation du client avec la version 1.6.0
client = Mistral(api_key=api_key)
print("Client Mistral initialisé.")

# Choix du modèle et définition des messages
model = "mistral-large-latest"  # Vous pouvez adapter le modèle selon vos besoins
messages = [
    {"role": "user", "content": "Quel est le meilleur fromage français ?"}
]

# Appel non-streaming de la fonction chat via la méthode complete
chat_response = client.chat.complete(
    model=model,
    messages=messages,
)

# Affichage de la réponse générée par le modèle
print("Réponse du modèle :", chat_response.choices[0].message.content)

In [ ]:
import os
import numpy as np
from mistralai import Mistral
from mistralai.models import EmbeddingRequest
from sklearn.metrics.pairwise import cosine_similarity

# Remplacez par votre vraie clé API
api_key = ""

# Initialisation du client avec la version 0.4.2
client = Mistral(api_key=api_key)
print("Client Mistral initialisé.")

# Textes pour lesquels nous générons des embeddings
textes_mistral = [
    "Quelles sont les heures d'ouverture de la mairie ?",
    "La mairie de Trifouillis-sur-Loire est ouverte du lundi au vendredi de 8h30 à 17h00.",
    "Le marché hebdomadaire a lieu tous les samedis matin sur la place centrale."
]

# Modèle d'embedding de Mistral
model_mistral = "mistral-embed"


print(f"\nGénération des embeddings avec le modèle '{model_mistral}'...")

# Appel à l'API via la méthode create
response = client.embeddings.create(
    model=model_mistral,
    inputs=textes_mistral
)

# Extraction des embeddings de la réponse
embeddings_mistral_raw = response.data
# Conversion en array numpy pour faciliter les calculs (chaque objet a un attribut .embedding)
embeddings_mistral = np.array([item.embedding for item in embeddings_mistral_raw])

print("Embeddings Mistral générés !")
print("Shape du premier embedding Mistral :", embeddings_mistral[0].shape)

# --- Calcul de similarité ---
print("\n--- Calcul de Similarité (Mistral) ---")
embedding_question_mistral = embeddings_mistral[0]
embedding_doc1_mistral = embeddings_mistral[1]
embedding_doc2_mistral = embeddings_mistral[2]

similarity_q_doc1_mistral = cosine_similarity([embedding_question_mistral], [embedding_doc1_mistral])[0][0]
similarity_q_doc2_mistral = cosine_similarity([embedding_question_mistral], [embedding_doc2_mistral])[0][0]

print(f"Question : '{textes_mistral[0]}'")
print("-" * 30)
print(f"Document 1 : '{textes_mistral[1]}'")
print(f"Similarité avec la question : {similarity_q_doc1_mistral:.4f}")
print("-" * 30)
print(f"Document 2 : '{textes_mistral[2]}'")
print(f"Similarité avec la question : {similarity_q_doc2_mistral:.4f}")
print("-" * 30)

if similarity_q_doc1_mistral > similarity_q_doc2_mistral:
    print("\nConclusion (Mistral) : Le document 1 est sémantiquement plus proche de la question.")
else:
    print("\nConclusion (Mistral) : Le document 2 est sémantiquement plus proche de la question.")




## Conclusion et Prochaines Étapes

Nous avons vu deux approches pour générer des embeddings :

#### Hugging Face (sentence-transformers):

- Avantages: Gratuit (modèles open source), contrôle total sur le modèle, fonctionne hors ligne une fois téléchargé.

- Inconvénients: Nécessite des ressources locales (RAM, CPU/GPU), peut être moins performant que les modèles propriétaires de pointe.

#### Mistral AI (API):
- Avantages: Modèles très performants, pas de gestion d'infrastructure (serverless), facile à intégrer.

- Inconvénients: Coût basé sur l'utilisation (payant), nécessite une connexion internet, dépendance vis-à-vis du fournisseur.
